# Digits Encoding

FIDO URIを生成する際には、CBORとしてシリアライズされたデータを数字列に変換します。具体的には、7バイトを17桁の数字にエンコードします。

元のバイト列が7の倍数でない場合、1〜6バイトが余りますが、それぞれ3, 5, 8, 10, 13, 15桁の数字としてエンコードします。

このようなエンコーディング方法には特に名前が付けられていないようですが、これを「Digits Encoding」と呼ぶことにします。


In [1]:
import struct

def digit_encode(data: bytes) -> str:
    chunk_size = 7
    chunk_digits = 17
    zeros = "00000000000000000"

    ret = ""
    while len(data) >= chunk_size:
        chunk = data[:chunk_size] + b'\x00'  # Extend to 8 bytes
        value = int.from_bytes(chunk, 'little')
        v = str(value)
        ret += zeros[:chunk_digits - len(v)]
        ret += v
        data = data[chunk_size:]

    if len(data) != 0:
        partial_chunk_digits = 0x0fda8530
        digits = (partial_chunk_digits >> (4 * len(data))) & 0xF
        chunk = data + b'\x00' * (8 - len(data))
        value = int.from_bytes(chunk, 'little')
        v = str(value)
        ret += zeros[:digits - len(v)]
        ret += v

    return ret


In [2]:
def digit_decode(encoded: str) -> bytes:
    chunk_digits = 17
    decoded = b""
    while len(encoded) >= chunk_digits:
        chunk_str = encoded[:chunk_digits]
        value = int(chunk_str)
        chunk = value.to_bytes(8, 'little')[:7]  # Take only the first 7 bytes
        decoded += chunk
        encoded = encoded[chunk_digits:]

    if len(encoded) != 0:
        partial_chunk_digits = 0x0fda8530
        for i in range(1, 7):
            if len(encoded) == ((partial_chunk_digits >> (4 * i)) & 0xF):
                value = int(encoded)
                chunk = value.to_bytes(8, 'little')[:i]
                decoded += chunk
                break

    return decoded


# ラウンドトリップ確認
digit_encodeとdigit_decodeがバイナリデータに対してラウンドトリップを達成していることを確認。

In [3]:
data = b'1234567abcdef'
encoded = digit_encode(data)
print(f"Encoded: {encoded}")

decoded = digit_decode(encoded)
print(f"Decoded: {decoded}")
print(f"Original and decoded are the same: {data == decoded}")


Encoded: 15540725856023089112585661964897
Decoded: b'1234567abcdef'
Original and decoded are the same: True


# 実際のFIDO URIをデコード
実際にGithubにPasskeyでログインしようとしたときに生成されたFIDO URIの数字部分をデコードしてみる。
このデータはすでに期限を過ぎているためこれで認証することはできない。

In [4]:
decoded : bytes = digit_decode("144519942798050940878582488612106138031714230513094139379299368895081878828178926639100664952474023496122943190653681470073382838502067711648524250383106107096654083332")
decoded

b'\xa6\x00X!\x03X3ebeC\xa96\x1f\xc6\xad\xe8\x80\xa4\x1d\x87\xa4\xe6\xca2y\x841\xd79\xfd\xbd\xd7\xbfBH\xb6\x01P\xc5\\\x02S\x86\xa58\x9e\xad+\x05\x1b7\xbci\x9b\x02\x02\x03\x1afEb)\x04\xf5\x05bga'

In [5]:
import cbor2
cbor2.loads(decoded)

{0: b'\x03X3ebeC\xa96\x1f\xc6\xad\xe8\x80\xa4\x1d\x87\xa4\xe6\xca2y\x841\xd79\xfd\xbd\xd7\xbfBH\xb6',
 1: b'\xc5\\\x02S\x86\xa58\x9e\xad+\x05\x1b7\xbci\x9b',
 2: 2,
 3: 1715823145,
 4: True,
 5: 'ga'}

### デコードされたCBORオブジェクトの説明

```plaintext
{
  0: b'\x03X3ebeC\xa96\x1f\xc6\xad\xe8\x80\xa4\x1d\x87\xa4\xe6\xca2y\x841\xd79\xfd\xbd\xd7\xbfBH\xb6',
  1: b'\xc5\\\x02S\x86\xa58\x9e\xad+\x05\x1b7\xbci\x9b',
  2: 2,
  3: 1715823145,
  4: True,
  5: 'ga'
}
```

### キーと対応する値の詳細

1. **キー 0** (33バイトの圧縮公開鍵)
   - **値**: `b'\x03X3ebeC\xa96\x1f\xc6\xad\xe8\x80\xa4\x1d\x87\xa4\xe6\xca2y\x841\xd79\xfd\xbd\xd7\xbfBH\xb6'`
   - **説明**: 33バイトのP-256, X9.62形式の圧縮公開鍵。

2. **キー 1** (16バイトのランダムQR秘密鍵)
   - **値**: `b'\xc5\\\x02S\x86\xa58\x9e\xad+\x05\x1b7\xbci\x9b'`
   - **説明**: 16バイトのランダムなQR秘密鍵。

3. **キー 2** (トンネルサーバードメインの数)
   - **値**: `2`
   - **説明**: 実装が知っているトンネルサーバードメインの数。

4. **キー 3** (エポック秒での現在時刻)
   - **値**: `1715823145`
   - **説明**: 2024年5月15日 23:05:45 UTCに対応するエポック秒での時刻。

5. **キー 4** (状態アシスト取引の可能性)
   - **値**: `True`
   - **説明**: デバイスが状態アシスト取引を行えることを示すブール値。

6. **キー 5** (取引タイプのヒント)
   - **値**: `'ga'`
   - **説明**: 次に`getAssertion`が続くことを示すヒント。このヒントはユーザーにすぐにガイダンスを提供するためのものです。